In [ ]:
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import linemerge, unary_union, polygonize
import fiona
import json

# === Step 1: 加载 C4 路段数据 ===
c4_gdf = gpd.read_file("ringc4.geojson")

# 合并为单条路径 + 封闭成 Polygon
merged_lines = linemerge(unary_union(c4_gdf.geometry))
polygons = list(polygonize(merged_lines))

# 选出最大的闭合区域 Polygon（防止生成多个小环）
largest_polygon = max(polygons, key=lambda p: p.area)

# === Step 2: 加载 tokyo.geojson 并筛选落在 C4 区域内的道路 ===
filtered_features = []

with fiona.open("tokyo.geojson", 'r') as src:
    crs = src.crs  # 获取坐标系信息
    for feature in src:
        geom = shape(feature["geometry"])
        if geom.within(largest_polygon):  # 是否落在 C4 环内
            filtered_features.append({
                "type": "Feature",
                "geometry": json.loads(json.dumps(feature["geometry"])),
                "properties": feature["properties"]
            })

# === Step 3: 保存结果为新的 GeoJSON ===
filtered_geojson = {
    "type": "FeatureCollection",
    "features": filtered_features
}

with open("tokyo_inside_c4.geojson", "w", encoding="utf-8") as f:
    json.dump(filtered_geojson, f, ensure_ascii=False)

print("✅ 筛选完成！输出文件为 tokyo_inside_c4.geojson")
